In [31]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
from sklearn.utils import shuffle
from numpy import asarray

import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png

print('import done')

import done


In [32]:
dataset_train = "Dataset1/queensland/train.xlsx"
dataset_test = "Dataset1/queensland/test.xlsx"

In [33]:
train = pd.read_excel(dataset_train)
test = pd.read_excel(dataset_test)

train = train.sample(frac=1)
test = test.sample(frac=1)

print(train.head())
print('-'*30)
print(train['label'].value_counts())
print('-'*30)

# train = train[train['text'].str.len()  15]

# print('x'*50)
# # df[df['column name'].map(len) < 2]
# print(train.head())
# print('-'*30)
# print(train['label'].value_counts())


                tweet_id                                               text  \
4862  296017201721376000  alls good moment cyclones passed Our house str...   
5568  295774747986361984  Abbott maintains 2011 Queensland flood levy ba...   
2628  296848657167294016                  The club made 76 since 10am hahah   
3026  295744506236636992  Master Electricians Australia free hotline hel...   
2120  296608686137278016                               My life like ooh aah   

             label  
4862      relevant  
5568      relevant  
2628  not_relevant  
3026      relevant  
2120  not_relevant  
------------------------------
relevant        3248
not_relevant    2771
Name: label, dtype: int64
------------------------------


In [34]:
# train['text'][0]
# train['label'][0]
dataColumn = 'text'
labelColumn = 'label'

In [35]:
tags = train[labelColumn]
texts = train[dataColumn]

tags_Y = test[labelColumn]
texts_Y = test[dataColumn]

In [36]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags.astype(str))
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)


# For testing data
le = LabelEncoder()
tags_Y = le.fit_transform(tags_Y.astype(str))
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts_Y)
mat_texts = tok.texts_to_matrix(texts_Y,mode='count')

[1 1 0 1 0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6019,) (6019, 1000)


In [37]:
# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)



# For testing data
cnn_texts_seq_Y = tok.texts_to_sequences(texts_Y)
cnn_texts_mat_Y = sequence.pad_sequences(cnn_texts_seq_Y,maxlen=max_len)

[26, 554, 843, 376, 138, 425, 25]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0  26 554 843 376 138 425  25]
(6019, 100)


In [38]:
def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=12,verbose=1,validation_split=0.25)

In [1]:
def get_simple_model():
#     model = Sequential()
#     model.add(Dense(512, activation='relu', input_shape=(num_max,)))
#     model.add(Dropout(0.2))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))
#     model.summary()
#     model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['acc',metrics.binary_accuracy])

    model = Sequential()
    model.add(Dense(512, input_shape=(num_max,)))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
    print('compile done')
    return model

In [2]:
m1 = get_simple_model()
check_model(m1,mat_texts,tags)

NameError: name 'Sequential' is not defined

In [3]:
def get_cnn_model_v1():   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [4]:
m2 = get_cnn_model_v1()
model_to_png(m2, 'model_2')
# check_model(m2,cnn_texts_mat,tags)

NameError: name 'Sequential' is not defined

In [5]:
def get_cnn_model_v2(): # added embed   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        50, #!!!!!!!!!!!!!!!!!!!!!!!
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [6]:
m3 = get_cnn_model_v2()
model_to_png(m3, 'model_3')
# check_model(m3,cnn_texts_mat,tags)

NameError: name 'Sequential' is not defined

In [7]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        50,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [8]:
m4 = get_cnn_model_v3()
model_to_png(m4, 'model_4')
# check_model(m4,cnn_texts_mat,tags)

NameError: name 'Sequential' is not defined

In [9]:
def get_cnn_model_v4():    # Pre Trained Embeddings
    
    
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('Embeddings/glove.6B.100d.txt', encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    
#     create a weight matrix for words in training docs
    embedding_matrix = np.zeros((len(tok.word_index) + 1, 100))
    for word, i in tok.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    
    model = Sequential()    
    e = Embedding(len(tok.word_index) + 1, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
    model.add(e)  #!!!!!!!!!!!!!!!!!!!
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [10]:
m5 = get_cnn_model_v4()
model_to_png(m5, 'model_5')
# check_model(m5,cnn_texts_mat,tags)

FileNotFoundError: [Errno 2] No such file or directory: 'Embeddings/glove.6B.100d.txt'

In [11]:
scores = m3.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with Embedding')
print('-'*30)
scores = m4.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with Embedding and more Filters')
print('-'*30)
scores = m5.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with PRe Trained GLOVE Embedding')

NameError: name 'm3' is not defined